In [ ]:
from collections import Counter
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import requests
import json
import pandas as pd
import time

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


# Creating dataframe for all senators

In [ ]:
# Get list of all 100 senators
from bs4 import BeautifulSoup
import requests

url = 'https://www.socialseer.com/resources/us-senator-twitter-accounts/'
req = requests.get(url)
html = req.text
soup = BeautifulSoup(html, 'html.parser')

# Get twitter handles
body = soup.find('tbody')
td = body.find('td')
children = body.findChildren(recursive=False)
twitter_handles = []

# Loop through children
for child in children:
  if child.a:
    twitter_handles.append(child.a.text)

# Removing senators who no longer are in senate, or who changed their handles
remove = ['SenKamalaHarris', 'SenBennetCO', 'JeffFlake', 'SenJonKyl', 'SenateMajLdr', 'SenDougJones']
twitter_handles = [keep for keep in twitter_handles if keep not in remove]
twitter_handles.extend(['SenatorBennet', 'AlexPadilla4CA', 'SenatorSinema', 'SenMarkKelly', 'LeaderMcConnell', 'SenTuberville'])


In [ ]:
def list_text2(json_data, total_retrieved):

  sw_nltk = stopwords.words('english')
  string_appended = []
  for x in range(total_retrieved):
    for i in json_data['data'][x]:
      words = [word for word in i['text'].split() if word.lower() not in sw_nltk]
      new_text = " ".join(words)
      string_appended.append(new_text)

  str_empty = " "
  new_string = str_empty.join(string_appended)
  comma_separated = ''.join(new_string).split()

  count_words = Counter(comma_separated)
  return count_words

In [ ]:
# Get user id of each senator's username

bearer_token = 'AAAAAAAAAAAAAAAAAAAAAKWfUAEAAAAAVlRCaBC9G2XcfK12sEaYBSFMQFo%3D2Ras8AORAGYkWjjjZpicOkZWBShWS96mk74MFTDi6zYZLzqKAj'
headers = {'Authorization':('Bearer '+ bearer_token)}

def get_user_data(username):
  url = "https://api.twitter.com/2/users/by/username/" + username
  response = requests.request("GET", url, headers=headers)
  json_data_test = json.loads(response.text)
  print(json_data_test)
  return json_data_test['data']

# get_user_data("SenShelby")

In [ ]:
# Get tweets 

all_words_dictionary = {}
total_retrieved_global = 0

def get_tweets(username):
  global total_retrieved_global
  user_data = get_user_data(username)
  bearer_token = 'AAAAAAAAAAAAAAAAAAAAAKWfUAEAAAAAVlRCaBC9G2XcfK12sEaYBSFMQFo%3D2Ras8AORAGYkWjjjZpicOkZWBShWS96mk74MFTDi6zYZLzqKAj'
  headers = {'Authorization':('Bearer '+ bearer_token)}

  n = 5                        
  max_results = 100               
  total_retrieved = 0               
  next_token = ""        
  json_data = {'data': []}           

  # Gathering tweets
  while total_retrieved < n:

    if next_token == "":
      url = f"https://api.twitter.com/2/users/{user_data['id']}/tweets?exclude=retweets,replies&max_results={max_results}"
    else:
      url = f"https://api.twitter.com/2/users/{user_data['id']}/tweets?exclude=retweets,replies&max_results={max_results}&pagination_token={next_token}"

    # Extra paramteres
    # url += f"&tweet.fields=public_metrics,"
    
    # Make request to Twitter API enpoint - returns json object
    response = requests.request("GET", url, headers=headers)
    json_data_test = json.loads(response.text)
    json_data['data'].append(json_data_test['data'])

    # Retreive next_token in order to loop through more than 100 tweets at a time
    try:
      next_token = json_data_test['meta']['next_token']
    
    except:
      break
    total_retrieved += 1
    total_retrieved_global += 1

  # Getting unique words and counts of 200 manchin tweets from list_text() function
  words_in_tweet_dictionary = list_text2(json_data, total_retrieved)
  for word in words_in_tweet_dictionary.items():
    if word[0] not in all_words_dictionary:
      all_words_dictionary[word[0]] = 0
    all_words_dictionary[word[0]] += word[1]
  
  return user_data['username'], json_data, total_retrieved

In [ ]:
sw_nltk = stopwords.words('english')

def get_data_frame(sorted_word_list, all_tweets, n=60):
  feature_set = set(sorted_word_list[:n])
  
  instance_list = []
  for senator_tweets in all_tweets:
    for tweet_batch in senator_tweets[1]['data']:
      for tweet_data in tweet_batch:
        tweet = tweet_data["text"]
        word_count_dictionary = {word: 0 for word in feature_set}

        words = [word for word in tweet.split() if word.lower() not in sw_nltk] # get rid of unnecesary words
        new_text = " ".join(words)
        comma_separated = ''.join(new_text).split() # comma separated and cleaned tweet 

        for word in comma_separated:
          if word in feature_set:
            word_count_dictionary[word] += 1
        
        word_count_dictionary["author_twitter_handle"] = senator_tweets[0] # 'author_twitter_hanldes' is just a string that we expect won't ever appear in the a tweet and so we're using that for the author's name
        instance_list.append(word_count_dictionary)


  df = pd.DataFrame(columns=(sorted_word_list[:n] + ["author_twitter_handle"]), index=list(range(0, len(instance_list))))
  for i, instance in enumerate(instance_list):
    df.iloc[i] = instance

  return df

In [ ]:
# Scrape tweets
all_of_the_tweets = [get_tweets(handle) for handle in twitter_handles]

word_count_list = list(all_words_dictionary.items())
word_count_list.sort(reverse=True, key=(lambda item: item[1]))
sorted_word_list = [word[0] for word in word_count_list]

df = get_data_frame(sorted_word_list, all_of_the_tweets)
df.head()

In [ ]:
df.to_csv('word_count_60.csv')

# MLP Model - Dataset w/ 30 features

In [ ]:
import pandas as pd
df_wce = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/word_count_encoding.csv')

In [ ]:
df_wce.shape

(48637, 32)

In [ ]:
df_wce.drop(columns=['Unnamed: 0'], inplace=True)
pd.set_option('display.max_columns', 15) #set to None if you dont want an elipses 
df_wce.head(10)

,&amp;,help,American,Senate,need,work,get,...,one,continue,President,protect,Act,–,author_twitter_handle
0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,SenShelby
1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,SenShelby
2,0,0,0,0,0,0,0,...,0,0,0,1,0,0,SenShelby
3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,SenShelby
4,0,0,0,0,0,0,0,...,0,0,0,0,0,0,SenShelby
5,0,0,0,0,0,0,0,...,0,0,1,0,0,0,SenShelby
6,1,0,0,0,0,0,0,...,1,0,0,0,0,0,SenShelby
7,1,0,0,0,0,0,0,...,0,0,0,0,0,0,SenShelby
8,3,0,0,0,0,1,0,...,0,0,0,0,0,0,SenShelby
9,0,0,0,0,0,0,0,...,0,0,0,0,0,1,SenShelby


In [ ]:
from sklearn.model_selection import train_test_split

X = df_wce.drop(columns=['author_twitter_handle'])
y = df_wce.author_twitter_handle
# y = pd.get_dummies(y)



## Grid Search

In [ ]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/word_count_encoding.csv')
df.head()

,Unnamed: 0,&amp;,help,American,Senate,need,work,get,must,I’m,bill,support,make,people,Biden,health,Americans,time,U.S.,would,bipartisan,working,great,new,across,one,continue,President,protect,Act,–,author_twitter_handle
0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,SenShelby
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,SenShelby
2,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,SenShelby
3,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,SenShelby
4,4,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,SenShelby


In [ ]:
# Grid search
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
mlp_gs = MLPClassifier(max_iter=600)

parameter_space = {
    'hidden_layer_sizes': [(64,10),(64,)],
    'activation': ['relu', 'logistic'],
    'solver': ['sgd', 'adam'],
    'alpha': [0.0001, 0.05],
    'learning_rate': ['constant','adaptive'],
    'momentum': [0.9, 0.65]
}
from sklearn.model_selection import GridSearchCV
# clf = GridSearchCV(mlp_gs, parameter_space, n_jobs=-1, cv=5)
clf = GridSearchCV(mlp_gs, parameter_space, n_jobs=-1, cv=5)
clf.fit(X_train, y_train) 

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  UserWarning,


In [ ]:
print('Best parameters found:\n', clf.best_params_)

In [ ]:
y_true, y_pred = y_test , clf.predict(X_test)
from sklearn.metrics import classification_report
print('Results on the test set:')
print(classification_report(y_true, y_pred))

## MLP

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
mlp_sk = MLPClassifier(learning_rate='constant', learning_rate_init=0.1, hidden_layer_sizes=(60), momentum=0.65, activation='relu', max_iter=500,
                       n_iter_no_change=10, solver='adam').fit(X_train, y_train)

accuracy = mlp_sk.score(X_test, y_test)
epochs = mlp_sk.n_iter_

print(f'Test accuracy: {100*round(accuracy,3)}%')
print(f'Number of epochs: {epochs}')

Test accuracy: 5.800000000000001%
Number of epochs: 477


## KNN Classifier

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler

X_min_max = pd.DataFrame(MinMaxScaler().fit_transform(X))

X_train, X_test, y_train, y_test = train_test_split(X_min_max, y, test_size=0.2)

knn = KNeighborsClassifier(n_neighbors=60, weights='distance')
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
score = metrics.accuracy_score(y_test, y_pred)
print(f'Accuracy: \t {round(score, 5)*100}%')

Accuracy: 	 4.749%


## SVM Model

In [ ]:
from sklearn.svm import SVC

# X_train, X_test, y_train, y_test = train_test_split(X_min_max, y, test_size=0.2)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

SVC_model = SVC()
SVC_model.fit(X_train, y_train)
SVC_prediction = SVC_model.predict(X_test)
print(metrics.accuracy_score(SVC_prediction, y_test))

0.059004934210526314


## Decision Tree

In [ ]:
from sklearn import tree
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

clf_sk = tree.DecisionTreeClassifier(criterion='entropy', min_samples_leaf=2, min_impurity_decrease=.0001)

clf_sk = clf_sk.fit(X_train, y_train)

accuracy = clf_sk.score(X_test, y_test)
print(f'Accuracy of model: {round(accuracy,6)*100}%')

Accuracy of model: 4.4921999999999995%


In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=10, criterion='entropy', min_samples_leaf=2, min_impurity_decrease=.0001)
clf.fit(X_train, y_train)
accuracy = clf.score(X_test, y_test)
print(f'Accuracy of model: {round(accuracy,6)*100}%')

Accuracy of model: 4.9548000000000005%


# Word Count Encoding Pipeline

## Pipeline Methods

In [ ]:
# Data Processing Methods

def get_data(url, col_drop):
  import pandas as pd
  pd.set_option('display.max_columns', 20)
  df = pd.read_csv(url)
  for col in col_drop:
    try:
      df.drop(columns=col, inplace=True)
    except:
      pass
  return df

def drop_missing_columns(df, cutoff=0.55):
  import pandas as pd
  for col in df:
    if (df[col].isna().sum() / len(df)) >= cutoff:
      df.drop(columns=[col], inplace=True)
  return df


def binning_categorical(df, cutoff=0.10):
  import pandas as pd

  for col in df:
    if not pd.api.types.is_numeric_dtype(df[col]):
      for group, count in df[col].value_counts().iteritems():
        if count/len(df) < cutoff:
          df.loc[df[col] == group, col] = 'Other'
  return df


def bin_time(row):
    if row['time_day'] >= 0 and row['time_day'] < 7:
        return 'midnight'
    if row['time_day'] >= 7 and row['time_day'] < 12:
        return 'morning'
    if row['time_day'] >= 12 and row['time_day'] < 18:
        return 'afternoon'
    return 'night'


def dummy_encode(df):
  import pandas as pd
  for col in df:
    if col != 'author_name':
      if not pd.api.types.is_numeric_dtype(df[col]):
        return pd.get_dummies(df, columns=[col], drop_first=True)


def fix_time_day(df, column):
  df[column] = df.apply(lambda row: bin_time(row), axis=1)
  return df


def min_max_normalization(X):
  # Min max normalization
  from sklearn.preprocessing import MinMaxScaler
  from sklearn import preprocessing

  min_max_scaler = preprocessing.MinMaxScaler()
  return min_max_scaler.fit_transform(X)


def z_score_normalization(X):
  # Normailze dataframe
  from sklearn.preprocessing import StandardScaler
  from sklearn import preprocessing

  scaler = preprocessing.StandardScaler().fit(X)
  return scaler.transform(X)



In [ ]:
# Algorithms

def try_models(X,y):
  from sklearn.model_selection import train_test_split
  from sklearn import preprocessing
  from sklearn import metrics

  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

  fit = {}

  print('Trying MLP model')
  from sklearn.neural_network import MLPClassifier
  # mlp = MLPClassifier(learning_rate='constant', learning_rate_init=0.05, hidden_layer_sizes=(20), momentum=0.9, activation='relu', max_iter=1500,
  #                      n_iter_no_change=20, solver='sgd').fit(X_train, y_train)
  mlp = MLPClassifier(max_iter=500).fit(X_train, y_train)
  fit['MLP'] = [mlp.score(X_test, y_test), mlp]


  print('Trying KNN model')
  from sklearn.neighbors import KNeighborsClassifier
  knn = KNeighborsClassifier(n_neighbors=25, weights='distance')
  knn.fit(X_train, y_train)
  y_pred = knn.predict(X_test)
  fit['KNN'] = [metrics.accuracy_score(y_test, y_pred), knn]


  print('Trying SVC model')
  from sklearn.svm import SVC
  SVC_model = SVC()
  SVC_model.fit(X_train, y_train)
  SVC_prediction = SVC_model.predict(X_test)
  fit['SVC'] = [metrics.accuracy_score(SVC_prediction, y_test), SVC_model]


  print('Trying Decision Tree model')
  from sklearn import tree
  from sklearn.model_selection import train_test_split
  DT_clf = tree.DecisionTreeClassifier(criterion='entropy', min_samples_leaf=2, min_impurity_decrease=.0001)  
  DT_clf.fit(X_train, y_train)
  fit['DecisionTree'] = [DT_clf.score(X_test, y_test), DT_clf]


  print('Trying Random Forest model\n')
  from sklearn.ensemble import RandomForestClassifier
  RFC = RandomForestClassifier(n_estimators=10)
  RFC.fit(X_train, y_train)
  fit['RandomForest'] = [RFC.score(X_test, y_test), RFC]

  return fit


## Run Algorithms

In [ ]:
# Full test - Normal 30 features

df = get_data('/content/drive/MyDrive/Colab Notebooks/Data/word_count_encoding.csv', col_drop=['Unnamed: 0'])

X = df.drop(columns=['author_twitter_handle']) # dropping unnecesary columns
y = df['author_twitter_handle']


# # Deciding to normalize w/ min_max
# X = min_max_normalization(X)


fit = try_models(X, y)

accuracy = sorted(fit, key=fit.__getitem__, reverse=True)
for acc in accuracy:
  print(f'{acc}:\t{fit[acc][0]}')

Trying MLP model
Trying KNN model
Trying SVC model
Trying Decision Tree model
Trying Random Forest model

SVC:	0.06044407894736842
MLP:	0.058182565789473686
DecisionTree:	0.04615542763157895
RandomForest:	0.04584703947368421
KNN:	0.041426809210526314


In [ ]:
# Full test - 20 Features

df = get_data('/content/drive/MyDrive/Colab Notebooks/Data/word_count_encoding.csv', col_drop=['Unnamed: 0'])

X = df.drop(columns=['author_twitter_handle']) # dropping unnecesary columns
X = X[X.columns[:-10]]
y = df['author_twitter_handle']


# # Deciding to normalize w/ min_max
# X = min_max_normalization(X)


fit = try_models(X, y)

accuracy = sorted(fit, key=fit.__getitem__, reverse=True)
for acc in accuracy:
  print(f'{acc}:\t{fit[acc][0]}')

Trying MLP model


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


Trying KNN model
Trying SVC model
Trying Decision Tree model
Trying Random Forest model

SVC:	0.04399671052631579
MLP:	0.04358552631578947
DecisionTree:	0.03834292763157895
RandomForest:	0.038240131578947366
KNN:	0.028885690789473683


In [ ]:
# Full test - 10 Features

df = get_data('/content/drive/MyDrive/Colab Notebooks/Data/word_count_encoding.csv', col_drop=['Unnamed: 0'])

X = df.drop(columns=['author_twitter_handle']) # dropping unnecesary columns
X = X[X.columns[:-20]]
y = df['author_twitter_handle']


# # Deciding to normalize w/ min_max
# X = min_max_normalization(X)


fit = try_models(X, y)

accuracy = sorted(fit, key=fit.__getitem__, reverse=True)
for acc in accuracy:
  print(f'{acc}:\t{fit[acc][0]}')

Trying MLP model
Trying KNN model
Trying SVC model
Trying Decision Tree model
Trying Random Forest model

SVC:	0.030838815789473683
MLP:	0.03022203947368421
RandomForest:	0.02837171052631579
DecisionTree:	0.02826891447368421
KNN:	0.02045641447368421


More

In [ ]:
# Full test - Normal 60 features

df = get_data('/content/drive/MyDrive/Colab Notebooks/Data/word_count_60.csv', col_drop=['Unnamed: 0'])

X = df.drop(columns=['author_twitter_handle']) # dropping unnecesary columns
y = df['author_twitter_handle']


# # Deciding to normalize w/ min_max
# X = min_max_normalization(X)


fit = try_models(X, y)

accuracy = sorted(fit, key=fit.__getitem__, reverse=True)
for acc in accuracy:
  print(f'{acc}:\t{fit[acc][0]}')


Trying MLP model


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


Trying KNN model
Trying SVC model
Trying Decision Tree model
Trying Random Forest model

SVC:	0.08145402121305735
MLP:	0.06961178045515395
RandomForest:	0.05478323550612707
KNN:	0.05447430748635568
DecisionTree:	0.05272371537431778


In [ ]:
# Full test - Normal 50 features

df = get_data('/content/drive/MyDrive/Colab Notebooks/Data/word_count_60.csv', col_drop=['Unnamed: 0'])

X = df.drop(columns=['author_twitter_handle']) # dropping label
X = X[X.columns[:-10]]
y = df['author_twitter_handle']


fit = try_models(X, y)

accuracy = sorted(fit, key=fit.__getitem__, reverse=True)
for acc in accuracy:
  print(f'{acc}:\t{fit[acc][0]}')

Trying MLP model


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


Trying KNN model
Trying SVC model
Trying Decision Tree model
Trying Random Forest model

SVC:	0.07403974873854392
MLP:	0.0668314282772114
RandomForest:	0.051796931315003604
DecisionTree:	0.0474719390382041
KNN:	0.047265987025023166


In [ ]:
# Full test - Normal 40 features

df = get_data('/content/drive/MyDrive/Colab Notebooks/Data/word_count_60.csv', col_drop=['Unnamed: 0'])

X = df.drop(columns=['author_twitter_handle']) # dropping label
X = X[X.columns[:-20]]
y = df['author_twitter_handle']


fit = try_models(X, y)

accuracy = sorted(fit, key=fit.__getitem__, reverse=True)
for acc in accuracy:
  print(f'{acc}:\t{fit[acc][0]}')

Trying MLP model


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


Trying KNN model
Trying SVC model
Trying Decision Tree model
Trying Random Forest model

SVC:	0.07342189269900114
MLP:	0.0639481000926784
RandomForest:	0.05231181134795593
DecisionTree:	0.05025229121614664
KNN:	0.04819277108433735
